In [1]:
# Prepare graph data...
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import namedtuple
Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y'])
class GraphDataset():
    def __init__(self, input_dir, n_samples=None):
        input_dir = os.path.expandvars(input_dir)
        filenames = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.startswith('event') and f.endswith('.npz')]
        self.filenames = (
            filenames[:n_samples] if n_samples is not None else filenames)

    def __getitem__(self, index):
        return load_graph(self.filenames[index])

    def __len__(self):
        return len(self.filenames)
def get_dataset(input_dir,n_files):
    return GraphDataset(input_dir, n_files)
def load_graph(filename):
    graph_dir = os.path.join(os.getcwd(), 'graphs')
    # Construct the full path to the specified file
    full_path = os.path.join(graph_dir, filename)
    """Read a single graph NPZ"""
    with np.load(full_path) as f:
        return sparse_to_graph(**dict(f.items()))
def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.float32):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y)
#Function to load X, Ri, Ro, Y.
def load_raw(graph_name,xyr):
    graph_ex=load_graph(graph_name)
    #Load raw data
    y=graph_ex.y
    Ri=graph_ex.Ri
    Ro=graph_ex.Ro
    X=graph_ex.X
    if xyr=='X':
        return X
    elif xyr=='Ri':
        return Ri
    elif xyr=='Ro':
        return Ro
    else:
        return y
# Load X data of dimension Nv x 4(including color)
import os
import pandas as pd
dir_ = os.path.join(os.getcwd(), 'color')
#dir_ = os.path.join(os.getcwd(), 'coloredX')
file_path = os.path.join(dir_, "event000001000_g000.csv")
v = pd.read_csv(file_path)
print("features data loaded...")
#Convenient representation of edge data 
#edges = [[i1,j1], [i2,j2], ... ]; i1, i2,... are outgoing-nodes, and j1, j2, ... are incoming-nodes
import json
graph='event000001000_g000'
with open("./networks/"+graph+".json", "r") as json_file:
        _,_,edges= json.load(json_file)
print("edge data loaded...")

features data loaded...
edge data loaded...


In [2]:
g_name='event000001000_g000.npz'
Ri=load_raw(g_name,'Ri')
Ro=load_raw(g_name,'Ro')
Ri,Ro=Ri.T,Ro.T
Rio=Ri+Ro
Rio.shape
y=load_raw(g_name,'y')

In [3]:
import json
# Specify the path to your JSON file
json_file_path = 'event000001000_g000_pairs.json'
# Open the file and load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)
# Access the data
all_pairs=data["all_pairs"]
real_pairs,fake_pairs=data["real_pairs"],data["fake_pairs"]
quads,f_quads=data["quads"],data["f_quads"]

In [4]:
quads[0]

[[0, 1], [1, 2], [2, 3]]

In [12]:
X=(v[0:4].values).flatten()

In [52]:
import pennylane as qml
from qiskit.circuit import Parameter
# Create a quantum circuit with Nv qubits
Nv = 4
num_qubits = Nv * 4  # 4 qubits per row (x,y,z,color)
#layers=4
# Define a list of trainable parameters for the entanglement
num_params = Nv  # Number of trainable parameters for the entanglement
import torch
import torch.nn as nn
import torch_geometric.nn as geom_nn
import pennylane as qml
from pennylane import numpy as np
ansatz = torch.randn(Nv, requires_grad=True)
#ansatz = np.random.random(size=(num_params*layers))

In [53]:
def quantum_circuit(X,ansatz):
    for i in range(Nv):
        for j in range(4):
            if j==0:
                qml.RX(X[i*4+j], wires = i * 4 + j)
            elif j==1:
                qml.RY(X[i*4+j], wires = i * 4 + j)
            else:
                qml.RZ(X[i*4+j], wires = i * 4 + j)
        #Entangle 3 positions
        qml.CNOT(wires = [i * 4, i * 4 + 1])
        qml.CNOT(wires = [i * 4 + 1, i * 4 + 2])
        qml.CNOT(wires = [i * 4 + 2, i * 4])
        #Entangle color and position-x.
        qml.CNOT(wires = [i * 4 + 3, i * 4])
        # Encode each feature value into a qubit 
    # Apply trainable entanglement gates
    for i in range(Nv):
        qml.RZ(ansatz[i], wires = i * 4)
    #Fully connect
    for i in range(Nv):
        for j in range(i + 1, Nv):
            # Use trainable parameters for entanglement angles
            qml.CNOT(wires = [i * 4, j * 4])
# Updated quantum_layer function
@qml.qnode(qml.device("default.qubit", wires=4*Nv))
def quantum_layer(x,params):
    quantum_circuit(x,params)
    return [qml.expval(qml.PauliZ(i*4)) for i in range(Nv)]
# Create a custom quantum layer with PennyLane
class QuantumLayer(nn.Module):
    def __init__(self, num_qubits):
        super(QuantumLayer, self).__init__()
        self.num_qubits = num_qubits
    def forward(self, x, params):
        return quantum_layer(x,params)

In [108]:
class CustomMessagePassingLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(CustomMessagePassingLayer, self).__init__(aggr='add')
        self.linear = nn.Linear(in_channels * 2, out_channels)  # Input size is doubled for edge pairs

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # Concatenate node features for each edge
        message = torch.cat([x_i, x_j], dim=1)
        return self.linear(message)

    def update(self, aggr_out, x):
        return aggr_out + x

class ClassicalMessagePassing(nn.Module):
    def __init__(self, num_nodes, num_edges, num_node_features, num_edge_features):
        super(ClassicalMessagePassing, self).__init__()
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.num_node_features = num_node_features
        self.num_edge_features = num_edge_features

        self.message_passing_layer = CustomMessagePassingLayer(num_node_features, num_edge_features)
        self.edge_classifier = nn.Linear(num_node_features, 1)

    def forward(self, x, edge_index):
        # Classical message passing layer
        classical_output = self.message_passing_layer(x.view(self.num_nodes, -1), edge_index)
        # Temporary edge features by combining node features (e.g., addition)
        temp_edge_features = torch.tensor([])
        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                edge_f = classical_output[i] + classical_output[j]
                temp_edge_features = torch.cat((temp_edge_features, edge_f.unsqueeze(0)), dim=0)
        # Edge-level prediction
        edge_scores = self.edge_classifier(temp_edge_features)
        return edge_scores.squeeze()

# Example usage
num_nodes = 4
num_edges = 6
num_node_features = 3
num_edge_features = 1  # Scalar edge feature

# Assuming x and edge_index are your input features and edge connections
x = torch.randn(num_nodes, num_node_features)
edge_index = torch.tensor([[0, 0, 0, 1, 1, 2], [1, 2, 3, 2, 3, 3]])

model = ClassicalMessagePassing(num_nodes, num_edges, num_node_features, num_edge_features)
classical_output = model(x, edge_index)
print(classical_output)

tensor([ 1.8590,  1.4237,  0.8091,  0.8818,  0.2672, -0.1681],
       grad_fn=<SqueezeBackward0>)


In [119]:
# Define a model that integrates quantum and classical components
class QuantumMessagePassing(nn.Module):
    def __init__(self, num_qubits, quantum_params):
        super(QuantumMessagePassing, self).__init__()
        self.num_qubits = num_qubits
        self.quantum_params = quantum_params
        self.quantum_layer = QuantumLayer(num_qubits)
        self.message_passing_layer = CustomMessagePassingLayer(1, 1)  # Assuming 1-dimensional input for each node
        self.edge_classifier = nn.Linear(num_edge_features, 1)  # Input size is doubled for edge pairs
    def forward(self, x, edge_index):
        # Quantum layer
        quantum_output = self.quantum_layer(x, self.quantum_params)
        # Classical message passing layer
        #classical_output = self.message_passing_layer(x.view(-1, 1), edge_index).view(-1)  # Assuming 1-dimensional input for each node
        first_q=torch.tensor([quantum_output[n] for n in edge_index[0]])
        second_q=torch.tensor([quantum_output[n] for n in edge_index[1]])
        quantum_edge=first_q+second_q
        print(quantum_edge)
        # Combine quantum and classical outputs for each node in the edge pair
        combined_output = quantum_edge+classical_output
        combined_output = combined_output.float()
        combined_output = combined_output.reshape(6, 1)
        # Edge-level prediction
        edge_scores = self.edge_classifier(combined_output)
        return edge_scores.squeeze()

In [120]:
# Example usage
quantum_params = torch.randn(Nv, requires_grad=True)
model = QuantumMessagePassing(num_qubits, quantum_params)

# Assuming x and edge_index are your input features and edge connections
edge_index = torch.tensor([[0, 0, 0, 1, 1, 2], 
                           [1, 2, 3, 2, 3, 3]])  # Fully connected graph

output = model(X, edge_index)

tensor([1.0967, 1.1003, 1.1047, 0.8777, 0.8820, 0.8857], dtype=torch.float64)
